### Running a single-machine cluster
When run on a single machine, this is equivalent to the dask 'processes' scheduler, with the added addition of a handy status dashboard. It's accessible by web browser via the link below. (The local cluster server can be set up and run outside the notebook for persistence.)

In [1]:
from dask import distributed

CLUSTER_SETTINGS = dict(
    # limiting this can help to avoid system hangs caused by google drivefs
    n_workers=8,
    threads_per_worker=1,
    # host=f"tcp://127.0.0.1:8786",
    dashboard_address=f"127.0.0.1:8787",
)

# cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
if "scheduler" not in dir():
    cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
    scheduler = distributed.Client(cluster.scheduler_address)

cluster

LocalCluster(c949804e, 'tcp://127.0.0.1:51162', workers=8, threads=8, memory=31.73 GiB)

### Creating dask dataframe objects
The following assumes that you've downloaded the specified zip archive at the specified path.

In [8]:
from sea_ingest.dask_ops import log_to_json

# this is a hack for working inside the repo source tree;
# normally, for pip install, just `import sea_ingest`
import sea_ingest
from labbench import stopwatch
import pandas as pd
import typing
import dask

data_path = "data/2023-09-25_GMM.zip"
data_timezone = "America/Denver"


def capture_summary(partition_data: typing.Dict[str, pd.DataFrame]):
    """this could be expanded into a function that does more, but for now, just make a capture summary"""

    # partition_data is the result of running zipfile.read_seamf_zipfile on a subset of files in the zip archive.
    # it takes the form of a dictionary of pandas.DataFrame objects
    
    # The below assumes metadata version 4 or above when including the 
    # calibration temperature. Remove the temperature key if running
    # this code using older data.

    partition_data["capture_summary"] = pd.DataFrame.from_dict(
        {
            "median_rms_pfp": sea_ingest.trace(
                partition_data, "pfp", capture_statistic="mean", detector="rms"
            ).median(axis=1),
            "max_max_pfp": sea_ingest.trace(
                partition_data, "pfp", capture_statistic="max", detector="peak"
            ).max(axis=1),
            "median_mean_power": sea_ingest.trace(
                partition_data, "psd", capture_statistic="mean"
            ).median(axis=1),
            "max_max_power": sea_ingest.trace(
                partition_data, "psd", capture_statistic="mean"
            ).max(axis=1),
            "noise_figure": sea_ingest.trace(partition_data, "channel_metadata")[
                "cal_noise_figure_dB"
            ].astype("float16"),
            "gain": sea_ingest.trace(partition_data, "channel_metadata")[
                "cal_gain_dB"
            ].astype("float16"),
            "calibration_temperature": sea_ingest.trace(partition_data, "channel_metadata")[
                "cal_temperature_degC"
            ].astype("float16"),
        }
    )

    return partition_data

# overwrite
log_to_json("data.log")

with dask.config.set(scheduler=cluster.scheduler), stopwatch("setup"):
    # scan the zipfiles to map out the file contents. the resulting is a dictionary of dask dataframes.
    # this is fast, because it hasn't loaded much data yet; other operations trigger "compute" that can
    # take a while to scrape the data out of the zip archive
    ddfs = sea_ingest.read_seamf_zipfile_as_ddf(
        data_path,
        partition_func=capture_summary,
        partition_size=200,
        tz=data_timezone,
        localize=False,
    )

C:\Users\aromaniello\AppData\Local\pdm\pdm\Cache\packages\eliot-1.14.0-py2.py3-none-any\lib\eliot\json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):
C:\Users\aromaniello\AppData\Local\pdm\pdm\Cache\packages\dask-2023.5.0-py3-none-any\lib\dask\base.py:1368: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
 INFO   2023-10-12 14:16:07.812 • labbench: setup 0.668 s elapsed


In [9]:
with stopwatch("compute"):
    # head forces a compute operation
    df = ddfs["capture_summary"].head(10)
df

 INFO   2023-10-12 14:16:15.599 • labbench: compute 6.243 s elapsed


,frequency,median_rms_pfp,max_max_pfp,median_mean_power,max_max_power,noise_figure,gain,calibration_temperature
datetime,,,,,,,,
2023-09-17 17:59:06.672000+00:00,3.545000e+09,-99.2500,-83.68750,-169.250,-169.000,4.933594,29.843750,29.09375
2023-09-17 17:59:11.302000+00:00,3.555000e+09,-81.1875,-59.15625,-150.625,-147.750,4.601562,30.125000,29.09375
2023-09-17 17:59:17.219000+00:00,3.565000e+09,-80.8750,-60.12500,-150.000,-147.750,4.300781,30.546875,29.09375
2023-09-17 17:59:21.862000+00:00,3.575000e+09,-78.0000,-57.25000,-147.750,-144.875,4.308594,30.437500,29.09375
2023-09-17 17:59:26.955000+00:00,3.585000e+09,-76.3750,-56.31250,-146.625,-144.750,4.160156,30.609375,29.09375
2023-09-17 17:59:32.280000+00:00,3.595000e+09,-81.2500,-59.65625,-150.625,-147.750,4.164062,30.718750,29.09375
2023-09-17 17:59:36.955000+00:00,3.605000e+09,-77.1250,-57.50000,-147.625,-146.125,4.156250,30.734375,29.09375
2023-09-17 17:59:42.963000+00:00,3.615000e+09,-88.6250,-67.43750,-157.750,-155.750,4.234375,30.625000,29.09375
2023-09-17 17:59:47.656000+00:00,3.625000e+09,-86.3750,-67.43750,-156.500,-154.000,4.265625,30.421875,29.09375


#### on-demand access to a subset of the data
The dask .loc accessor allows faster access to a specified subset of the data

In [10]:
with stopwatch("compute"):
    df = (
        ddfs["capture_summary"]
        .loc["2023-09-21 20:50:00":"2023-09-21 20:51:00"]
        .compute(scheduler=scheduler)
    )

df

 INFO   2023-10-12 14:16:45.044 • labbench: compute 16.592 s elapsed


,frequency,median_rms_pfp,max_max_pfp,median_mean_power,max_max_power,noise_figure,gain,calibration_temperature
datetime,,,,,,,,
2023-09-21 20:50:03.672000+00:00,3.705000e+09,-58.15625,-40.87500,-129.000,-127.8125,5.449219,28.843750,28.203125
2023-09-21 20:50:13.521000+00:00,3.545000e+09,-99.25000,-86.62500,-169.250,-169.1250,4.902344,29.890625,28.203125
2023-09-21 20:50:18.148000+00:00,3.555000e+09,-78.87500,-59.53125,-148.750,-146.6250,4.574219,30.171875,28.203125
2023-09-21 20:50:24.139000+00:00,3.565000e+09,-78.25000,-60.90625,-148.000,-146.6250,4.277344,30.593750,28.203125
2023-09-21 20:50:28.811000+00:00,3.575000e+09,-74.50000,-56.75000,-144.875,-143.3750,4.281250,30.484375,28.203125
2023-09-21 20:50:33.911000+00:00,3.585000e+09,-76.37500,-57.40625,-146.500,-145.1250,4.125000,30.656250,28.203125
2023-09-21 20:50:39.263000+00:00,3.595000e+09,-81.50000,-60.06250,-150.750,-148.2500,4.132812,30.750000,28.203125
2023-09-21 20:50:43.943000+00:00,3.605000e+09,-79.37500,-58.18750,-149.500,-146.5000,4.128906,30.781250,28.203125
2023-09-21 20:50:49.887000+00:00,3.615000e+09,-89.93750,-69.62500,-159.875,-157.1250,4.203125,30.671875,28.203125


#### Bulk data
Dask dataframes support data save operations split output files by partition. This means that the time span of each output file can be adjusted with 'repartition'. In this example, a file save function is applied to each partition.

In [14]:
import dask
from pathlib import Path


def write_feather(df: pd.DataFrame, dirpath):
    """example for feather, but could substitute code for e.g. csv, database, etc. instead"""
    path = Path(dirpath) / df.index[0].strftime("%Y-%m-%d.feather")

    # feather like most formats requires string column names
    df.columns = df.columns.astype("str")
    df.reset_index().to_feather(path, compression="zstd")
    return str(path)


def write_pickle(df: pd.DataFrame, dirpath):
    """example for pickles"""

    path = Path(dirpath) / df.index[0].strftime("%Y-%m-%d.p.zstd")
    df.to_pickle(path)
    return str(path)


# collecting the write operations into a single options
# allows dask to optimize the execution so that the zip archives
# only need to be read once.
#
# the choice of dictionary means that the returned list of
# files written will be returned as a dictionary with the same keys.
#
# Note that running the code below requires directories "capture_summary"
# and "pfp" to exist within your "data" directory.
ddf = ddfs["capture_summary"]

save_ops = dict(
    capture_summary=(
        ddfs["capture_summary"]
        .repartition(freq="1MS")
        .map_partitions(write_pickle, "data/capture_summary")
    ),
    pfp=(ddfs["pfp"].repartition(freq="1W").map_partitions(write_pickle, "data/pfp")),
)

with stopwatch("compute"):
    files_saved = dask.compute(save_ops, scheduler=scheduler)

    print("wrote the following files:")
    print(files_saved)

 INFO   2023-10-12 14:25:46.141 • labbench: compute 47.410 s elapsed


wrote the following files:
({'capture_summary': 0    data\capture_summary\2023-09-17.p.zstd
dtype: object, 'pfp': 0    data\pfp\2023-09-17.p.zstd
dtype: object},)
